In [2]:
import openpyxl
import pandas as pd
import numpy as np
from numpy.linalg import norm

feature_names = ['accelerations',	'fetal_movement',	'uterine_contractions',	'light_decelerations',	'severe_decelerations',	'prolongued_decelerations',
              'abnormal_short_term_variability',	'mean_value_of_short_term_variability',	'percentage_of_time_with_abnormal_long_term_variability',
              'mean_value_of_long_term_variability',	'histogram_width',	'histogram_min',	'histogram_max',	'histogram_number_of_peaks',
              'histogram_number_of_zeroes',	'histogram_mode',	'histogram_mean',	'histogram_median',	'histogram_variance',	'histogram_tendency']
noOfIterations = 20

def convertSign(value):
    if value == "positive":
        return 1
    elif value == "negative":
        return 0
    else: 
        return -1

def generateSigns(feature_names):
    dataframe = openpyxl.load_workbook("outputFetalKernelShapLOOCV_1.xlsx")
    sheet = dataframe.active
   
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_KernelShap = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 6
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[3].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 3
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_KernelShap = pd.DataFrame(dict)
     
    df_KernelShap.to_excel("signsFetalKernelShap.xlsx")
   
    #################################################################
    dataframe = openpyxl.load_workbook("outputFetalTreeShapLOOCV_1.xlsx")
    sheet = dataframe.active
   
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_TreeShap = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 6
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[3].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 3          
          
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_TreeShap = pd.DataFrame(dict)
    
    df_TreeShap.to_excel("signsFetalTreeShap.xlsx")    

    ############################################################################
    dataframe = openpyxl.load_workbook("outputFetalLIMELOOCV_20.xlsx")
    sheet = dataframe.active
    
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_LIME = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 10
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[5].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 5            
            
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_LIME = pd.DataFrame(dict)
   
    df_LIME.to_excel("signsFetalLIME.xlsx")
   

def stackRowsColumns(M, noOfRows, noOfColumns):
    pos = [0] * noOfRows
    for i in range (0, noOfRows):
        pos[i] = i
    
    for i in range(1, noOfRows):
        wsim = [0] * noOfRows
        for j in range(i, noOfRows):
            for k in range(0, i):
                A = M.loc[k]
                B = M.loc[j]
                wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
        max_value = max(wsim)
        max_index = wsim.index(max_value)
        aux = M.loc[max_index]
        M.loc[max_index] = M.loc[i]
        M.loc[i] = aux
        aux = pos[max_index]
        pos[max_index] = pos[i]
        pos[i] = aux
    
    for i in range(1, noOfColumns):
        wsim = [0] * noOfColumns
        for j in range(i, noOfColumns):
            for k in range(0, i):
                A = M.loc[k]
                B = M.loc[j]
                wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
        max_value = max(wsim)
        max_index = wsim.index(max_value)
        aux = M.loc[max_index]
        M.loc[max_index] = M.loc[i]
        M.loc[i] = aux
    return pos, M

def cosineSimilarity(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

def similarityCase(M, i):
    s = 0
    k = i - 1
    for j in range (1, k + 1):
        s = s + cosineSimilarity(M.loc[i], M.loc[i - j]) * (1 / j)
    return s

def similarityMatrix(M, noOfRows):
    s = 0
    for i in range(1, noOfRows):
        s = s + similarityCase(M, i)
    s = s / (noOfRows - 1)
    return s

def GlobalAlign(MSP, MS, noOfRows):
    if similarityMatrix(MSP, noOfRows) >  similarityMatrix(MS, noOfRows):
        max = similarityMatrix(MSP, noOfRows)
        min = similarityMatrix(MS, noOfRows)
    else:
        max = similarityMatrix(MS, noOfRows)
        min = similarityMatrix(MSP, noOfRows)
    return min / max
    #return similarityMatrix(MSP, noOfRows) / similarityMatrix(MS, noOfRows)


def createMSP(MP, MS, noOfRows):
    MS_copy = MS.copy(deep = True)
    MP_copy = MP.copy(deep = True)
    posS, MS_copy = stackRowsColumns(MS_copy, noOfRows, len(feature_names))
    posP, MP_copy = stackRowsColumns(MP_copy, noOfRows, len(feature_names))
    # MSP = pd.DataFrame(columns = feature_names)
    # for i in range(0, noOfRows):
    #     MSP.loc[posP[i]] = MS_copy.loc[posS[i]]
    return GlobalAlign(MP_copy, MS_copy, noOfRows)

generateSigns(feature_names)
df_KernelShap = pd.read_excel('signsFetalKernelShap.xlsx')
df_KernelShap = df_KernelShap[df_KernelShap.columns[1:]]
df_TreeShap = pd.read_excel('signsFetalTreeShap.xlsx')
df_TreeShap = df_TreeShap[df_TreeShap.columns[1:]]
df_LIME = pd.read_excel('signsFetalLIME.xlsx')
df_LIME = df_LIME[df_LIME.columns[1:]]

results = pd.DataFrame(columns = ['LL', 'LS', 'SL', 'SS'])
noOfInstances = df_KernelShap.shape[0]

for instance in range(0, noOfInstances):
    result = []
    data_Shap = []
    for i in range(int(noOfIterations / 2)):
        data_Shap.append(df_KernelShap.loc[instance].values.flatten().tolist())
        data_Shap.append(df_TreeShap.loc[instance].values.flatten().tolist())
    M_Shap  = pd.DataFrame(data_Shap, columns = feature_names)
    
    data_LIME = []
    for i in range(noOfIterations):
        data_LIME.append(df_LIME.loc[instance * noOfIterations + i].values.flatten().tolist())
    M_LIME  = pd.DataFrame(data_LIME, columns = feature_names)
    
    MP = M_LIME
    MS = M_LIME
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_LIME
    MS = M_Shap
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_Shap
    MS = M_LIME
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_Shap
    MS = M_Shap
    result.append(createMSP(MP, MS, noOfIterations))
    print(result)
    results.loc[instance] = result


results.to_excel("resultsSignsFetalMPMS_MinMax.xlsx")

[1.0, 0.8283321438322128, 0.8283321438322128, 1.0]
[1.0, 0.8884101039165655, 0.8884101039165655, 1.0]
[1.0, 0.8394904544326777, 0.8394904544326777, 1.0]
[1.0, 0.7440216077722744, 0.7440216077722744, 1.0]
[1.0, 0.7275853976723171, 0.7275853976723171, 1.0]
[1.0, 0.9686289254359917, 0.9686289254359917, 1.0]
[1.0, 0.8556999049805483, 0.8556999049805483, 1.0]
[1.0, 0.8619870101324152, 0.8619870101324152, 1.0]
[1.0, 0.8719658428078165, 0.8719658428078165, 1.0]
[1.0, 0.851841959088025, 0.851841959088025, 1.0]
[1.0, 0.7858949443453807, 0.7858949443453807, 1.0]
[1.0, 0.7676883285548857, 0.7676883285548857, 1.0]
[1.0, 0.7350455631300934, 0.7350455631300934, 1.0]
[1.0, 0.8917063939067345, 0.8917063939067345, 1.0]
[1.0, 0.9513382349816227, 0.9513382349816227, 1.0]
[1.0, 0.8305923219448265, 0.8305923219448265, 1.0]
[1.0, 0.8623944255072827, 0.8623944255072827, 1.0]
[1.0, 0.9508683846596226, 0.9508683846596226, 1.0]
[1.0, 0.7739673039677009, 0.7739673039677009, 1.0]
[1.0, 0.9128646631900114, 0.91286

C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_12388\3852202452.py:124: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_12388\3852202452.py:140: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_12388\3852202452.py:149: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.777887316707971, 0.777887316707971, 1.0]
[1.0, 0.6912703056930674, 0.6912703056930674, 1.0]
[1.0, 0.6948058889475401, 0.6948058889475401, 1.0]
[1.0, 0.8385075664369022, 0.8385075664369022, 1.0]
[1.0, 0.6507931215653454, 0.6507931215653454, 1.0]
[1.0, 0.6456658195940861, 0.6456658195940861, 1.0]
[1.0, 0.665151658189767, 0.665151658189767, 1.0]
[1.0, 0.6907667308696834, 0.6907667308696834, 1.0]
[1.0, 0.7171507512687919, 0.7171507512687919, 1.0]
[1.0, 0.6923817932540012, 0.6923817932540012, 1.0]
[1.0, 0.6735132480490293, 0.6735132480490293, 1.0]
[1.0, 0.7019537883950367, 0.7019537883950367, 1.0]
[1.0, 0.6709500376930423, 0.6709500376930423, 1.0]
[1.0, 0.6073628543443818, 0.6073628543443818, 1.0]
[1.0, 0.9781515880360351, 0.9781515880360351, 1.0]
[1.0, 0.6301388108764836, 0.6301388108764836, 1.0]
[1.0, 0.8915215273268977, 0.8915215273268977, 1.0]
[1.0, 0.6159665542232293, 0.6159665542232293, 1.0]
[1.0, 0.6482031017118125, 0.6482031017118125, 1.0]
[1.0, 0.803943